<a href="https://colab.research.google.com/github/nikhilsai27/NLP/blob/main/NLP_Exp_10_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
text_corpus = """
Evelyn Thorne, a world-famous violinist in her 50s, was preparing for the biggest concert of her career – a solo performance at Carnegie Hall. But beneath her success lay a past filled with tragedy.

Years ago, a devastating fire took the life of her younger sister, Lily, who was also a talented violinist. Evelyn, the only survivor, found solace in her music but was haunted by the memory of her sister.

Enter Sophia, Evelyn's 18-year-old daughter, also gifted in violin. Sophia resented her mother's constant push for perfection and the shadow of her late aunt, Lily, that loomed over her.

As the pressure mounted for the Carnegie Hall performance, a series of events forced Evelyn and Sophia to confront their painful past. A mysterious fire destroyed Evelyn's prized violin, a gift from Lily. A surprise visit from an old mentor revealed a long-held secret about the night of the fire. Sophia received an invitation to a prestigious music school, which could finally allow her to step out of her mother's shadow.

Facing her own mortality and the consequences of her choices, Evelyn embarked on a journey of reconciliation with Sophia. They delved into their shared history, uncovering new truths about Lily's life and the fire.

The climax came with Carnegie Hall looming. Sophia struggled with the weight of expectations and her complicated feelings towards her mother. Evelyn, vulnerable and raw, had to confront her deepest fears and the ghost of her past.

In the end, they found a way to mend their broken bond and connect through their shared passion for music. Sophia chose her own path, free from the pressure of her mother's legacy. The climactic performance at Carnegie Hall became a poignant testament to the power of music to heal and the enduring love between a mother and daughter.
"""

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_corpus])
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in text_corpus.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

In [6]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

In [8]:
learning_rate = 0.01
batch_size = 50
epochs = 10

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [10]:
model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/10
6/6 [==============================] - 5s 329ms/step - loss: 5.1022 - accuracy: 0.0302
Epoch 2/10
6/6 [==============================] - 2s 286ms/step - loss: 4.7786 - accuracy: 0.0638
Epoch 3/10
6/6 [==============================] - 1s 162ms/step - loss: 4.5871 - accuracy: 0.0671
Epoch 4/10
6/6 [==============================] - 1s 167ms/step - loss: 4.3041 - accuracy: 0.1141
Epoch 5/10
6/6 [==============================] - 1s 178ms/step - loss: 3.8423 - accuracy: 0.1946
Epoch 6/10
6/6 [==============================] - 1s 175ms/step - loss: 3.2185 - accuracy: 0.3020
Epoch 7/10
6/6 [==============================] - 1s 203ms/step - loss: 2.4676 - accuracy: 0.4128
Epoch 8/10
6/6 [==============================] - 1s 182ms/step - loss: 1.7275 - accuracy: 0.6007
Epoch 9/10
6/6 [==============================] - 1s 170ms/step - loss: 1.0543 - accuracy: 0.7886
Epoch 10/10
6/6 [==============================] - 1s 156ms/step - loss: 0.6377 - accuracy: 0.9262


In [11]:
import numpy as np

def generate_text(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probs = model.predict(token_list)[0]
        predicted_word_index = np.random.choice(len(predicted_probs), p=predicted_probs)
        output_word = tokenizer.index_word.get(predicted_word_index, "")
        seed_text += " " + output_word
    return seed_text

In [12]:
generated_text = generate_text("Generated Text:", 10, model, max_sequence_length)
print(generated_text)

1/1 [==============================] - 0s 31ms/step
Generated Text: ago a world haunted violinist violinist in her 50s preparing


In [13]:
val_loss, val_accuracy = model.evaluate(X, y, verbose=0)
print(f"Loss: {val_loss:.4f}")
print(f" Accuracy: {val_accuracy:.4f}")

Loss: 0.3762
 Accuracy: 0.9732
